# **Step 1**

**Task: Import	data	from	"BSCY4.csv"**

Import Data from given CSV file using Pandas

In [2]:
import numpy as np 
import pandas as pd

data = pd.read_csv('BSCY4.csv')
print(data.dtypes)


Unnamed: 0      float64
Date             object
AveragePrice     object
Total Volume    float64
4046            float64
4225            float64
4770            float64
Total Bags      float64
Small Bags      float64
Large Bags      float64
XLarge Bags     float64
type             object
year            float64
region           object
dtype: object


# **Step 2**

**Task: Cleanse	information	in	the	"date".	Do	all	rows	follow	the	
same	 format	when	it	comes	to	"date"? What	 formats	are	there	
and	how	many	entries	per	each	format?**

Based on the output data below we can see that the column 'Date' has three formats

1) day - month - year is taken from the column 'year' <br>
2) year - month - day <br>
3) day - month - year <br>


In [42]:
data.groupby(by=['Date','year']).size().sample(5)

Date        year  
05/02       2017.0     1
18/01       2015.0     1
29-10-2017  2017.0     1
2015-11-29  2015.0    52
14/05       2017.0     1
dtype: int64

Now we change the column 'Date'type into 'datetime' so we can change its format into the following format (%d-%m-%Y).

In [43]:
# Because not all items have the format %d-%m-%Y it will throw an error.
# So we will force the action and make the wrongly formated into NaN

newDate = pd.to_datetime(data['Date'], format="%d-%m-%Y", errors='coerce')

In [44]:
# The function below will switch the first(%Y) and last(%d)
def switch(input):

    input[0], input[-1] = input[-1], input[0]

    newDate = '-'.join(input)
    return newDate
    

In [45]:
# With the function above we only have one worngly formated date (%d-%m).
# The function below will get the year from the corresponding column 'year' and reformat it


def getYear(input, index):
    # get year 
    year = data['year'][index]
    
    # since objects in year has the '.0' at the end, we will remove it to make it look cleaner
    input.append(str(year).rstrip('0').rstrip('.'))
    
    return input


In [46]:
# we will get the index of all the NaN values
index = data[newDate.isnull() == True].index

# create a new column to insert formated values into
data['fValue'] = ''

# lop through the indexes
for i in index:
    
    # retrieve the value from 'Date'
    xDate = data['Date'][i]
    
    # remove '-' using split method and gain access to each value
    s = xDate.split('-')
    
    # if the lenght of value is not equals to 3 it is the value of format (%d-%m)
    if len(s) == 3:
        data.at[i, 'fValue'] = switch(s)
        
    # if the lenght of value is not equals to 3 it is the value of format (%d-%m)
    elif len(s) != 3:
        
        s2 = xDate.split('/')
        newS2 = '-'.join(getYear(s2, i))
        data.at[i, 'fValue'] = newS2    

# replace data['Date'] with newDate
data['Date'] = newDate

# fill the null values with the new column we created earlier called 'fValue'
data['Date'] = data['Date'].fillna(data['fValue'])

# Drop the 'fValue' as we dont need it anymore
data.drop('fValue', axis=1, inplace=True)

data['Date'].head()


0    14-01-2018
1    07-01-2018
2    27-12-2015
3    20-12-2015
4    13-12-2015
Name: Date, dtype: object

# **Step 3**

**Task: Cleanse	 the	 data	in	 the	 field	 "type".	How	many	 genuine	
categories	 are	 present?	 Do	 you	 see	 problems	 with	 how	 the	
categories	 represented?	 How	 many	 entries	 have	 errors?**

- The column 'type' contains 2 genuine categories called 'conventional' and 'organic'. <br>


In [47]:
# The issue on this column are the values 'Org.'
# The column 'type' has a total of 169 'Org.' values

data['type'].astype(str).str.contains('Org.').sum()

169

In [48]:
# We will now replace all of the 'Org.' into 'organic'
data['type'] = data['type'].astype(str).str.replace('Org.','organic')
data['type'] = data['type'].astype('category')

# Check if method
data['type'].astype(str).str.contains('Org.').sum()


0

# **Step 4**

**Task: Cleanse	 the	 content	 of	 the	 field	 "average	 price".	 How many	 genuine	 missing	 values	 are	 there?	 How	 many	 entries have	erroneous	string-based	representation.**

- The column 'Average Price' are of type float meaning it contains a dot ie. '12.2' <br>
- The error on this column is that some value have commas instead of a dot. <br>
- Another error is that the columns contains NaN


In [49]:
# Total Number of values that uses acomma in column 'Average Price' 
data['AveragePrice'].astype(str).str.contains(',').sum()

30

In [50]:
# Total Number of NaN in column 'Average Price' 
data['AveragePrice'].isna().sum()

20

In [51]:
# To cleanse the column we change all commas into dots
data['AveragePrice'] = data['AveragePrice'].astype(str).str.replace(',','.')

# we double check if method above works
data['AveragePrice'].astype(str).str.contains(',').sum()


0

In [52]:
# we will also replace all the NaN into 0 so that it is more appropriate rathar than leaving it as NaN
data['AveragePrice'] = data['AveragePrice'].astype(str).str.replace('nan','0')

# Dobule check that method above works
data['AveragePrice'].astype(str).str.contains('nan').sum()


0

In [56]:
# Change Type
data['AveragePrice'] = data['AveragePrice'].astype(np.float) 

# Cahnge type to DateTime
data['Date'] = pd.to_datetime(data['Date'])

data.dtypes

Unnamed: 0             float64
Date            datetime64[ns]
AveragePrice           float64
Total Volume           float64
4046                   float64
4225                   float64
4770                   float64
Total Bags             float64
Small Bags             float64
Large Bags             float64
XLarge Bags            float64
type                  category
year                   float64
region                  object
dtype: object

In [55]:
# Now we will export the 'sqlDataFrame' as a CSV file to be used on Data Consolidation 
csv = data.to_csv('BSCY4_csv.csv')
